<img src="../idp.jpg">

# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Aula 6 - Data Visualization</p>

<img src="datahackers2.jpeg">

## Objetivo do trabalho

Utilizar os conceitos, técnicas e novas formas de visualização de dados conforme vimos na nossa aula teórica.

Hoje vamos trabalhar com o maior e mais completo panorama sobre o mercado de trabalho brasileiro na área de dados. Veja mais detalhes no site da pesquisa: www.stateofdata.com.br - No site da pesquisa você também encontra um relatório completo onde exploramos esse dataset e chegamos a conclusões interessantes sobre o mercado de trabalho brasileiro na área de dados.

No período entre 18 de outubro de 2021 e 6 de dezembro de 2021, a maior comunidade de dados do Brasil,o <font color="#023469"><b>Data Hackers</b></font> juntamente com a consultoria global *Bain & Company* realizaram uma pesquisa por meio da aplicação de um questionário online, cujo objetivo é <b>mapear o mercado profissional de dados brasileiro</b>. 
A presente pesquisa é o resultado de um esforço conjunto da Data Hackers, a maior comunidade de dados do Brasil, e da Bain & Company, consultoria global que ajuda empresas e organizações a promover mudanças que definam o futuro dos negócios, para mapear o mercado de trabalho de dados no Brasil.


Em Maio de 2022, foram abertos os dados da pesquisa, para que a comunidade pudesse realizar suas próprias análises, concorrendo a prêmios, mas também podendo contar com o privilégio de grandes profissionais da área como jurados dessa competição.


## Fonte de dados


Link dos dados: https://www.kaggle.com/datasets/datahackers/state-of-data-2021  

O dataset foi anonimizado com o objetivo de garantir a privacidade dos respondentes, para isso foi necessário em alguns casos remover outliers que poderiam identificar o entrevistado e, portanto, nem todos os dados coletados na pesquisa estarão disponíveis aqui. Estados com menor incidência de resposta, como aqueles das regiões Norte, terão apenas sua região indicada no dataset, também como consequência do processo de anonimização, o mesmo aconteceu em algumas outras perguntas.

As perguntas cujas respostas são multi-valoradas ocupam mais de uma coluna no dataset. Portanto, para diferenciar quais colunas pertencem a quais perguntas, cada coluna é identificada com uma tupla. Sendo o primeiro identificador o da pergunta, e, no caso de várias respostas, o segundo identificador referencia a alternativa escolhida. As perguntas mapeadas são mostradas abaixo (lembrando que algumas foram removidas e outras tiveram alguns outliers transformados/apagados no processo de anonimização).

Parte 1 - Dados demográficos  
Parte 2 - Dados sobre carreira  
Parte 3 - Desafios dos gestores de times de dados  
Parte 4 - Conhecimentos na área de dados  
Parte 5 - Objetivos na área de dados  
Parte 6 - Conhecimentos em Engenharia de Dados/DE  
Parte 7 - Conhecimentos em Análise de Dados/DA  
Parte 8 - Conhecimentos em Ciências de Dados/DS  
Parte 9 - Sobre a comunidade Data Hackers  


## Nesse notebook faremos 2 análises com visualização de dados
1 - Análise visual (storytelling) com dados sobre a faixa salarial, região do país, senioridade e conhecimentos na área de dados  
2 - Análise visual (storytelling) explorando a desigualdade de gênero na área de dados

<hr/>

# 1. Análise Visual utilizando storytelling

Este projeto vislumbra examinar apenas uma fatia dessas dimensões, tendo como objeto de estudo a variável ('P2_h ', 'Faixa salarial') dentro de um recorte regional, por senioridade e conhecimentos na área de dados.

<img src="dataviz01.png">

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)

In [ ]:
colunas_selecionadas = ["('P1_e_b ', 'Regiao onde mora')",
 "('P1_g_c ', 'Mudou de Estado?')",
"('P2_a ', 'Qual sua situação atual de trabalho?')",
 "('P2_g ', 'Nivel')",
 "('P2_h ', 'Faixa salarial')",
 "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')",
 "('P4_d ', 'Quais das linguagens listadas abaixo você utiliza no trabalho?')",
 "('P4_f ', 'Quais dos bancos de dados/fontes de dados listados abaixo você utiliza no trabalho?')",
 "('P4_g ', 'Quais das opções de Cloud listadas abaixo você utiliza no trabalho?')",
 "('P4_h ', 'Quais as Ferramentas de Business Intelligence você utiliza no trabalho?')",                   
]

colunas_renomeadas = [
'regiao_mora',
 'mudou_estado',
'situacao_trabalho',
 'nivel',
 'faixa_salarial',
 'xp_dados',
 'linguagens_utilizadas',
'banco_dados_utilizados',
 'cloud_utilizadas',
 'ferramentas_bi_utilizadas',
]

trabalham = ['Empreendedor ou Empregado (CNPJ)',
 'Empregado (CLT)',
 'Estagiário',
 'Freelancer',
 'Servidor Público',
 'Trabalho na área Acadêmica/Pesquisador',
 'Vivo fora do Brasil e trabalho para empresa de fora do Brasil',
 'Vivo no Brasil e trabalho remoto para empresa de fora do Brasil (PJ)']


pesquisa2021 = pd.read_csv('datahackers.csv',low_memory=False,usecols=colunas_selecionadas)

#renomeando colunas
pesquisa2021.columns = colunas_renomeadas


#Removendo profissioanais que se consideram desempregados, apenas estudantes ...
pesquisa2021 = pesquisa2021[pesquisa2021["situacao_trabalho"].isin(trabalham)].reset_index(drop=True)

#Removendo profissionais que trabalham no exterior:
pesquisa2021 = pesquisa2021[pesquisa2021["regiao_mora"] != 'Exterior'].reset_index(drop=True)


print(f'Quantidade de registros:  {len(pesquisa2021)}')

In [ ]:
def ordena_respostas(respostas,sep):
    try: 
        respostas = sorted([res.strip() for res in respostas.split(sep)])
        respostas = ' - '.join(respostas).strip()
        return respostas
    except:
        pass

In [ ]:
import matplotlib
from matplotlib import transforms, pyplot as plt

%matplotlib inline

# definição da paleta de cores
GRAY1, GRAY2, GRAY3 = '#231F20', '#414040', '#555655'
GRAY4, GRAY5, GRAY6 = '#646369', '#76787B', '#828282'
GRAY7, GRAY8, GRAY9 = '#929497', '#A6A6A5', '#BFBEBE'
BLUE1, BLUE2 = '#174A7E', '#4A81BF'
RED1, RED2 = '#C3514E', '#E6BAB7'
GREEN1, GREEN2 = '#0C8040', '#9ABB59'


# definição de algumas fontes para os gráficos
plt.rcParams['font.family'] = 'Arial'
matplotlib.rcParams['mathtext.fontset'] = 'custom'
matplotlib.rcParams['mathtext.bf'] = 'Arial:bold'
matplotlib.rcParams['mathtext.it'] = 'Arial:italic'

## Descoberta e insights

Vamos tentar responder várias perguntas e contar histórias com os dados utilizando a técnica de storytelling e construindo gráficos diferenciados.

Entre as descobertas realizadas, teremos:

- D1 - O começo da vida nômade  
- D2 - Qual o principal destino desses profissionais?  
- D3 - Faixa salarial: diferenças regionais e por senioridade  
- D4 - Principais tecnologias vs Faixa salarial  
- D5 - Python e SQL reinam absolutos  
- D6 - Qual banco de dados devo aprender primeiro?  
- D7 - Quem ganha? AWS, Azure ou Google Cloud?  
- D8 - PowerBI Google Studio ou Tableau?  


## D1 - O começo da vida nômade

- É o que os dados dizem ao olharmos para os respondentes que afirmaram ter mudado de estado de origem.

- Observa-se um percentual relevante.

- **Nota**: exclui-se desse espaço amostral, respondentes que apenas estudam, ou não trabalham ou residem no exterior

In [ ]:
coluna_mudou_de_estado = pesquisa2021["mudou_estado"]
dist_mudou_de_estado = coluna_mudou_de_estado.value_counts()
percent_mudou_de_estado = 100*(dist_mudou_de_estado[1]/sum(dist_mudou_de_estado))


fig = plt.figure(figsize=(7.45, 4.9),
           dpi=110)            

# remove chart border
for spine in plt.gca().spines.values():
    spine.set_visible(False)

ax=plt.gca()
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

plt.text(-0.15, 1.03,
         'Profissionais mudam de cidade'+' '*27,
         fontsize=26,
         color='white',

         bbox={'facecolor': GRAY7, 'pad': 10, 'edgecolor': 'none'})

plt.text(-0.15, 0.04,
         '*desconsidera respondentes que se consideram desempregados, apenas estudantes, e que preferiram não informar a situação atual de trabalho',
         fontsize=10.5,
         color=GRAY3)

plt.text(-0.12, 0.77,
         'Entre os profissionais de dados que trabalham*,',
         fontsize=19,
         color=BLUE2)

plt.text(-0.15, 0.41,
     f'$\\bf{percent_mudou_de_estado:.2f}\%$', # use mathtext \\bf for bold text
         fontsize=72,
         color=BLUE2)

plt.text(-0.15, 0.25,
         # use mathtext \\bf for bold text
         '$\\bf{mudaram\ de\ estado\ de\ origem.}$',
         fontsize=19,
         color=BLUE2);


## D2 - Qual o principal destino desses profissionais?

In [ ]:
mudaram_de_estado = pesquisa2021[pesquisa2021["mudou_estado"] == 1].copy()
ranking_migracao = mudaram_de_estado.groupby(["regiao_mora"])["regiao_mora"].count().sort_values(ascending=False)

regioes = list(ranking_migracao.keys())
dist_regioes = list(ranking_migracao)
percent_regioes = [(dist*100)/sum(dist_regioes) for dist in dist_regioes]

# create new figure
fig, ax1 = plt.subplots(figsize=(15,15),  # width, height in inches
                        dpi=110)             # resolution of the figure

# tune the subplot layout by setting sides of the figure
fig.subplots_adjust(left=0.1, right=0.3, top=0.3, bottom=0.1)

# draw horizontal bars
ax1.barh(range(len(regioes)),
         percent_regioes[::-1],
         color=[GRAY9,GRAY7,GRAY5,GRAY3,GRAY1])


# set properties for axes object
plt.setp(ax1,
         xticks=[0, 20, 40, 60],  # 5 x-ticks only 0 and 1
         xticklabels=['0%', '20%', '40%', '60%'],  # with n% labels
         yticks=np.arange(len(regioes)),  # tick for all response
         yticklabels=regioes[::-1])  # with text labels

# change the appearance of ticks, tick labels, and gridlines 
ax1.tick_params(top='on', bottom='off', left='off', 
                labelbottom='off', labeltop='on')



# remove chart border
ax1.tick_params(color=GRAY7)
ax1.spines['top'].set_color(GRAY7)
ax1.spines['left'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.spines['bottom'].set_visible(False)


plt.text(0, 6,
         'A $\\bf{ Região\ Sudeste}$'
             ' é o principal destino\n'
             'entre os profissionais que mudam de estado',
         fontsize=15,
         color=GRAY1);

## D3 - Faixa salarial: diferenças regionais e por senioridade

In [ ]:
# Manipulação dos dados para construir o gráfico
top_salario_regiao = pesquisa2021.groupby(["regiao_mora"])["faixa_salarial"].describe().sort_values(by='freq',ascending=False)
count = list(top_salario_regiao['count'])
freq = list(top_salario_regiao['freq'])
percent = [round((count_/sum(count))*100,2) for count_ in count]
top_salario_regiao['percent'] = percent

#top_salario_regiao
regiao_norte = top_salario_regiao.T['Norte']['percent']
regiao_nordeste = top_salario_regiao.T['Nordeste']['percent']
regiao_centro_oeste = top_salario_regiao.T['Centro-oeste']['percent']
regiao_sudeste = top_salario_regiao.T['Sudeste']['percent']
regiao_sul = top_salario_regiao.T['Sul']['percent']

#faixa predominate por regiao
faixa_regiao_norte = top_salario_regiao.T['Norte']['top']
faixa_regiao_nordeste = top_salario_regiao.T['Nordeste']['top']
faixa_regiao_centro_oeste = top_salario_regiao.T['Centro-oeste']['top']
faixa_regiao_sudeste = top_salario_regiao.T['Sudeste']['top']
faixa_regiao_sul = top_salario_regiao.T['Sul']['top']

In [ ]:
top_salario_nivel_regiao = {}

for regiao in set(pesquisa2021["regiao_mora"]):
    salario_nivel_regiao = pesquisa2021[pesquisa2021["regiao_mora"] == regiao].groupby(["nivel"])["faixa_salarial"].describe().sort_values(by='freq',ascending=False).T
    top_salario_nivel_regiao[regiao] = [salario_nivel_regiao['Júnior']['top'],salario_nivel_regiao['Pleno']['top'],salario_nivel_regiao['Sênior']['top']]
    
dfTopSalarioNivelRegiao = pd.DataFrame(data=top_salario_nivel_regiao, index=['Júnior','Pleno','Sênior'])

#fixar a ordem das colunas das regiões
dfTopSalarioNivelRegiao = dfTopSalarioNivelRegiao[['Norte','Nordeste','Centro-oeste','Sudeste','Sul']]

Fazendo um corte por senioridade e região, a Faixa salarial predominante apresenta algumas particularidades:
Em contraste de possuir os maiores salários, ao olharmos por nível, a região Sudeste apresenta menor salário no nível Júnior.

No nível Pleno, as regiões Sul e Sudeste possuem maiores salários, entre 6k e 8k/mês.

Centro-Oeste possui o maior salário do nível Sênior, chegando a pagar entre 12k e 16k/mês.

In [ ]:
# Vamos chamar a atenção para essas descobertas afirmados acima.
dfTopSalarioNivelRegiao.style.applymap(lambda x: "background-color: orange", subset=pd.IndexSlice[('Júnior'),'Sudeste'])

In [ ]:
dfTopSalarioNivelRegiao.style.applymap(lambda x: "background-color: orange", subset=pd.IndexSlice[('Pleno'),'Sudeste':'Sul'])

In [ ]:
dfTopSalarioNivelRegiao.style.applymap(lambda x: "background-color: orange", subset=pd.IndexSlice[('Sênior'),'Centro-oeste'])

## D4 - Principais tecnologias vs Faixa salarial

- Quais são as tecnologias gerais mais utilizadas na área de dados?
- Quais devo estudar primeiro?

In [ ]:
def top_tecnologia_faixa_predominante(tecnologia):
    
    pesquisa2021[tecnologia] = pesquisa2021[tecnologia].apply(ordena_respostas,sep=',')
    df_top_tecnologia_faixa_predominante = pesquisa2021.groupby([tecnologia])['faixa_salarial'].describe().sort_values(by="count",ascending=False)[:10]
    count = list( df_top_tecnologia_faixa_predominante['count'])
    freq = list( df_top_tecnologia_faixa_predominante['freq'])
    percent = [round((count_/sum(count))*100,2) for count_ in count]
    df_top_tecnologia_faixa_predominante['percent'] = percent
    
    tecnologias = list(df_top_tecnologia_faixa_predominante.T)
    dist_tecnologias = list(df_top_tecnologia_faixa_predominante['percent'])
    faixa_predominante = list(df_top_tecnologia_faixa_predominante['top'])
    
    return tecnologias,dist_tecnologias,faixa_predominante

In [ ]:
def grafico_principais_tecnologias(tecnologias,dist_tecnologias,titulo,subtitulo,faixa_predominante):
    
    X = tecnologias[::-1]
    Y = dist_tecnologias[::-1]

    fig, ax1 = plt.subplots(figsize=(15,15), # width, height in inches
                            dpi=110)           # resolution of the figure


    fig.subplots_adjust(left=0.1, right=0.3, top=0.3, bottom=0.1)


    bars = ax1.barh(range(len(X)),
                    Y,
                    height=0.65,
                    color=[GRAY9,GRAY7,GRAY5,GRAY3,GRAY1])


    for i, b in enumerate(bars):
        plt.text(b.get_width() + 1, b.get_y() + 0.15,
                 f'{str(Y[i])}%',
                 fontsize=13,
                 color='black')


    plt.setp(ax1, yticks=np.arange(len(X)), yticklabels=X)


    ax1.tick_params(bottom='off', left='off', labelbottom='off')


    for item in ax1.get_yticklabels():
        item.set_fontsize(12.5)
        item.set_color(GRAY3)


    for spine in ax1.spines.values():
        spine.set_visible(False)

    # title the plot
    ax1.text(-10,6, titulo, fontsize=19, color=GRAY1)
    ax1.text(0,5.2, subtitulo,
             fontsize=14, color=GREEN1, fontweight='bold')


    plt.text(
       65, 4.5,
        'Faixa Salarial Predominante',size=12, color=BLUE1);
    
    plt.text(
       65, 3.8,
        f'{faixa_predominante[0]}',size=12, color=GRAY1);
    
    plt.text(
       65, 2.9,
        f'{faixa_predominante[1]}',size=12, color=GRAY3);
    
    plt.text(
       65, 2.0,
        f'{faixa_predominante[2]}',size=12, color=GRAY5);
    
    plt.text(
       65, 1.1,
        f'{faixa_predominante[3]}',size=12, color=GRAY7);
    
    plt.text(
       65, 0.2,
        f'{faixa_predominante[4]}',size=12, color=GRAY9);

## D5 - Python e SQL reinam absolutos

- Python e SQL de longe, são as linguagens de programação mais utilizadas.

- Um verdadeiro arroz e feijão de todo profissional de dados.

- Ao selecionarmos este grupo, encontrou-se a faixa salarial predominante entre 8k e 12k/mês


In [ ]:
linguagens, dist_linguagens, linguagens_top_faixa = top_tecnologia_faixa_predominante('linguagens_utilizadas')

grafico_principais_tecnologias(tecnologias=linguagens[:5],dist_tecnologias=dist_linguagens[:5],
                               titulo='Principais linguagens utilizadas',subtitulo='Python e SQL reinam absolutos',
                               faixa_predominante=linguagens_top_faixa)

## D6 - Qual banco de dados devo aprender primeiro?

- SQL Server liderou o ranking, entre os bancos de dados mais utilizados.

- Ao observarmos por faixa salarial, ganhamos a seguinte pista...

- Talvez seja interessante começar com SQL Server, porém aprender Googke BigQuery trás maiores oportunidades

In [ ]:
bancos, dist_bancos, bancos_top_faixa = top_tecnologia_faixa_predominante('banco_dados_utilizados')

grafico_principais_tecnologias(tecnologias=bancos[:5],dist_tecnologias=dist_bancos[:5],
                               titulo='Principais bancos de dados utilizados',subtitulo='SQL Server é o principal BD.\nAprender Google BigQuery parece promissor.',
                               faixa_predominante=bancos_top_faixa)

## D7 - Quem ganha? AWS, Azure ou Google Cloud?

- AWS é bastante popular e os dados da pesquisa nos comprovam isso.

- Contudo, adicionalmente aprender mais adiante Google Cloud pode levar a maiores salários.

In [ ]:
clouds, dist_clouds, clouds_top_faixa = top_tecnologia_faixa_predominante('cloud_utilizadas')

grafico_principais_tecnologias(tecnologias=clouds[:5],dist_tecnologias=dist_clouds[:5],
                               titulo='Principais clouds utilizadas',subtitulo='AWS pode ser uma boa oportunidade para aprender cloud.',
                               faixa_predominante=clouds_top_faixa)

## D8 - PowerBI Google Studio ou Tableau?

- Se você já ficou esperto até aqui, provavelmente sabe o que vou dizer.

- PowerBI tem se demonstrado bastante popular no mercado de dados.

- Porém, o profissional que aprende Tableau, pode atingir maiores salários.

In [ ]:
bi, dist_bi, bi_top_faixa = top_tecnologia_faixa_predominante('ferramentas_bi_utilizadas')

grafico_principais_tecnologias(tecnologias=bi[:5],dist_tecnologias=dist_bi[:5],
                               titulo='Principais ferramentas de Business Intelligence',subtitulo='PowerBI ainda é carro chefe em BI',
                               faixa_predominante=bi_top_faixa[:5])

Não tenho muita experiência em dados, e agora?
Tudo tem saída. Mesmo entre os profissionais que não possuem experiência na área, conseguem atingir salários dentro da média de entrada júnior do mercado.

In [ ]:
xp_faixa_salarial = pesquisa2021.groupby(['xp_dados'])['faixa_salarial'].describe().sort_values(by="count",ascending=False)

In [ ]:
xp_faixa_salarial.T.style.applymap(lambda x: "background-color: orange", subset=pd.IndexSlice[('top'),'Não tenho experiência na área de dados'])

**Em resumo, aprendemos nesse Storytelling que :**

- A região sudeste é o principal destino de quem se muda, e possui os melhores salários

- Entre as linguagens que você deve dominar, definitivamente aprende Python e SQL o mais rápido possível.

- Você pode começar a estudar por SQL Server, mas já mirar em aprender Google BigQuery mais adiante

- AWS é a cloud popular, mas novamente o produto da Google pode oferecer melhores oportunidades de salários

- PowerBi e Tableau são suas amigas para Business Intelligence

<hr/>

# 2. Análise Visual (storytlelling) explorando a desigualdade de gênero na área de dados
 
* Foram feitas análises de comparação dos gêneros, ligando os resultados as **metáforas da desigualdade**;
* Os gráficos são construidos utilizando o pacote plotly, que são interativos.
* Homens são maioria na área;
* Sobre os salários, há predominância (propocional) de mulheres nos salários mais baixos e de homens nos mais altos;
* Homens tem cargos mais altos e mais tempo de experiência;
* Balanceando fatores como nível educacional, nível de senioridade, cargo e tempo de experiência, a diferença salarial deixa de ser identificável.
* Em um primeiro olhar o resultado é positivo: ou seja, se tivermos as mesmas condições para homens e mulheres (tempo de experiência, cargos e formações mais requeridas), as mulheres podem alcançar as maiores faixas salariais também.
* Então, para que a igualdade salarial seja atingida: é preciso entender porque mulheres não alcançam os mesmos espaços que homens e ser proativos em ações que mudem esse cenário.

In [ ]:
#importando pacotes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
#importando dados
df = pd.read_csv('datahackers.csv', low_memory=False)

Pra facilitar a leitura e compreensão dos resultados, foram feitas algumas edições necessárias no banco de dados. 

*   Exclusão de individuos que definiram o gênero como "Outro", que apesar de ser um grupo muito importante para a análise de gênero, ainda não traz uma população grande o suficiente para uma análise estatística adequada. Além disso, pode facilitar a identificação de indivíduos.
*   Reordenação de algumas categorias
*   Agrupamento e renomeação variáveis

Além disso, para facilitar a leitura do código, foi criada uma função para geração de um tipo de gráfico mais utilizado nesta análise.




In [ ]:
# Removendo linhas em que o gênero é igual a "outro"
df = df[df["('P1_b ', 'Genero')"] != "Outro"]

# Atualizando a variável  "('P2_g ', 'Nivel')" da seguinte forma
# Quando a pessoa respondeu que é gestor "('P2_d ', 'Gestor?')" e não respondeu a pergunta "('P2_g ', 'Nivel')", 
# assume-se que o nível é "Gestor"

df.loc[(df["('P2_d ', 'Gestor?')"] == 1) & (df["('P2_g ', 'Nivel')"].isna()), "('P2_g ', 'Nivel')"] = 'Gestor'


# Atualizando a variável  "('P1_h ', 'Nivel de Ensino')"  da seguinte forma:
# Quando a pessoa  que ´refere não informar, define-se que a resposta foi None
df["('P1_h ', 'Nivel de Ensino')"] = df["('P1_h ', 'Nivel de Ensino')"].replace({'Prefiro não informar': None})

# Renomeando variáveis para que o nome fique mais curto e viável de inserir nos gráficos
df = df.rename(columns=({
"('P4_f_a ', 'MySQL')"                        : "MySQL",
"('P4_f_b ', 'Oracle')"                       : "Oracle",
"('P4_f_c ', 'SQL SERVER')"                   : "SQL SERVER",
"('P4_f_d ', 'SAP')"                          : "SAP",
"('P4_f_e ', 'Amazon Aurora ou RDS')"         : "Amazon Aurora ou RDS",
"('P4_f_f ', 'Amazon DynamoDB')"              : "Amazon DynamoDB",
"('P4_f_g ', 'CoachDB')"                      : "CoachDB",
"('P4_f_h ', 'Cassandra')"                    : "Cassandra",
"('P4_f_i ', 'MongoDB')"                      : "MongoDB",
"('P4_f_j ', 'MariaDB')"                      : "MariaDB",
"('P4_f_k ', 'Datomic')"                      : "Datomic",
"('P4_f_l ', 'S3')"                           : "S3",
"('P4_f_m ', 'PostgreSQL')"                   : "PostgreSQL",
"('P4_f_n ', 'ElasticSearch')"                : "ElasticSearch",
"('P4_f_o ', 'DB2')"                          : "DB2",
"('P4_f_p ', 'Microsoft Access')"             : "Microsoft Access",
"('P4_f_q ', 'SQLite')"                       : "SQLite",
"('P4_f_r ', 'Sybase')"                       : "Sybase",
"('P4_f_s ', 'Firebase')"                     : "Firebase",
"('P4_f_t ', 'Vertica')"                      : "Vertica",
"('P4_f_u ', 'Redis')"                        : "Redis",
"('P4_f_v ', 'Neo4J')"                        : "Neo4J",
"('P4_f_x ', 'Google BigQuery')"              : "Google BigQuery",
"('P4_f_y ', 'Google Firestore')"             : "Google Firestore",
"('P4_f_z ', 'Amazon Redshift')"              : "Amazon Redshift",
"('P4_f_z1 ', 'Amazon Athena')"               : "Amazon Athena",
"('P4_f_z2 ', 'Snowflake')"                   : "Snowflake",
"('P4_f_z3 ', 'Databricks')"                  : "Databricks",
"('P4_f_z4 ', 'HBase')"                       : "HBase",
"('P4_f_z5 ', 'Presto')"                      : "Presto",
"('P4_f_z6 ', 'Splunk')"                      : "Splunk",
"('P4_f_z7 ', 'SAP HANA')"                    : "SAP HANA",
"('P4_f_z8 ', 'Hive')"                        : "Hive",
"('P4_f_z9 ', 'Firebird')"                    : "Firebird"}))

#Renomeando categorias de variáveis para que o nome fique mais curto e viável de inserir nos gráficos

df["('P2_h ', 'Faixa salarial')"] = df["('P2_h ', 'Faixa salarial')"].replace({'Menos de R$ 1.000/mês' : "R&#36;1k",
                                             'de R$ 1.001/mês a R$ 2.000/mês'  : "R&#36;1k-2k",
                                             'de R$ 2.001/mês a R$ 3000/mês'   : "R&#36; 2k-3k",
                                             'de R$ 3.001/mês a R$ 4.000/mês'  : "R&#36; 3k-4k", 
                                             'de R$ 4.001/mês a R$ 6.000/mês'  : "R&#36; 4k-6k",
                                             'de R$ 6.001/mês a R$ 8.000/mês'  : "R&#36; 6k-8k",
                                             'de R$ 8.001/mês a R$ 12.000/mês' : "R&#36; 8k-12k",
                                             'de R$ 12.001/mês a R$ 16.000/mês' : "R&#36; 12k-16k",
                                             'de R$ 16.001/mês a R$ 20.000/mês' : "R&#36; 16k-20k",
                                             'de R$ 20.001/mês a R$ 25.000/mês' : "R&#36; 20k-25k",
                                             'de R$ 25.001/mês a R$ 30.000/mês' : "R&#36; 25k-30k",
                                             'de R$ 30.001/mês a R$ 40.000/mês' : "R&#36; 30k-40k",
                                             'Acima de R$ 40.001/mês' : "R&#36; 40k"                                      
                                                    })

df["('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"] = \
    df["('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"]\
    .replace({'Não tenho experiência na área de dados':'Não tenho experiência <br />na área de dados'})

df["('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')"] = \
            df["('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')"]. \
            replace({'Não tive experiência na área de TI/Engenharia de Software antes de começar a trabalhar na área de dados' :
                         'Não tive experiência<br /> em TI antes' })

df["('P2_f ', 'Cargo Atual')"] = df["('P2_f ', 'Cargo Atual')"].replace({ 
    "Cientista de Dados/Data Scientist"                       : "Cientista de Dados"
  , "Analista de BI/BI Analyst/Analytics Engineer"            : "Analista de BI"
  , "Analista de Dados/Data Analyst"                          : "Analista de Dados"
  , "Engenheiro de Dados/Data Engineer"                       : "Engenheiro de Dados"
  , "Outro"                                                   : "Outro"
  , "Analista de Negócios/Business Analyst"                   : "Analista de Negócios"
  , "Desenvolvedor ou Engenheiro de Software"                 : "Dev ou Engenheiro de Software"
  , "Engenheiro de Machine Learning/ML Engineer"              : "Engenheiro de Machine Learning"
  , "Outras Engenharias (não inclui dev)"                     : "Outras Engenharias (não inclui dev)"
  , "Estatístico"                                             : "Estatístico"
  , "Analista Administrativo"                                 : "Analista Administrativo"
  , "Analista de Inteligência de Mercado/Market Intelligence" : "Analista de Inteligência de Mercado"
  , "DBA/Administrador de Banco de Dados"                     : "Administrador de Banco de Dados"
  , "Analista de Sistemas/Analista de TI"                     : "Analista de Sistemas"
  , "Analista de Marketing"                                   : "Analista de Marketing"
  , "Técnico"                                                 : "Técnico"
  , "Suporte Técnico"                                         : "Suporte Técnico"
  , "Professor"                                               : "Professor"
  , "Arquiteto de Dados"                                      : "Arquiteto de Dados"
  , "Product Manager"                                         : "Product Manager"
  , "Arquiteto de dados"                                      : "Arquiteto de Dados" })

df["('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"] = df["('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"].astype('category')
df["('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"] =  \
  df["('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"].cat.rename_categories(
                                                         {'Ainda não temos pessoas atuando com dados na empresa': '0' })


#Reordenando categorias de variáveis para deixar em uma ordem que faça mais sentido na apresentação gráfica
df["('P1_h ', 'Nivel de Ensino')"] = df["('P1_h ', 'Nivel de Ensino')"].astype('category')
df["('P1_h ', 'Nivel de Ensino')"] = df["('P1_h ', 'Nivel de Ensino')"].cat\
                                     .reorder_categories(['Não tenho graduação formal', 
                                                          'Estudante de Graduação',
                                                          'Graduação/Bacharelado', 
                                                          'Pós-graduação',
                                                          'Mestrado',
                                                          'Doutorado ou Phd'
                                                           ])
df["('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"] = \
    df["('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"].astype('category')

df["('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"] = \
    df["('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')"].cat\
    .reorder_categories(['Não tenho experiência <br />na área de dados', 
                           'Menos de 1 ano', 
                           'de 1 a 2 anos', 
                            'de 2 a 3 anos', 
                           'de 4 a 5 anos', 
                           'de 6 a 10 anos', 
                           'Mais de 10 anos'])
                                     

df["('P2_c ', 'Numero de Funcionarios')"] = df["('P2_c ', 'Numero de Funcionarios')"].astype('category')
df["('P2_c ', 'Numero de Funcionarios')"] = df["('P2_c ', 'Numero de Funcionarios')"].cat\
                                            .reorder_categories(['de 1 a 5', 
                                                                 'de 6 a 10',
                                                                 'de 11 a 50', 
                                                                 'de 51 a 100', 
                                                                 'de 101 a 500',
                                                                 'de 501 a 1000', 
                                                                 'de 1001 a 3000', 
                                                                 'Acima de 3000'])


df['faixa_salarial'] = df["('P2_h ', 'Faixa salarial')"].astype('category')
df['faixa_salarial'] = df['faixa_salarial'].cat.reorder_categories(["R&#36;1k",
                                                                    "R&#36;1k-2k", "R&#36; 2k-3k",
                                                                    "R&#36; 3k-4k", "R&#36; 4k-6k",
                                                                    "R&#36; 6k-8k", "R&#36; 8k-12k",
                                                                    "R&#36; 12k-16k", "R&#36; 16k-20k",
                                                                    "R&#36; 20k-25k", "R&#36; 25k-30k",
                                                                    "R&#36; 30k-40k", "R&#36; 40k" ])



df["('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')"] = \
        df["('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')"].astype('category')

df["('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')"] = \
  df["('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')"]. \
  cat.reorder_categories(['Não tive experiência<br /> em TI antes',
                          'Menos de 1 ano', 
                          'de 1 a 2 anos', 
                          'de 2 a 3 anos', 
                          'de 4 a 5 anos', 
                          'de 6 a 10 anos', 
                          'Mais de 10 anos'])


df["('P2_g ', 'Nivel')"] = df["('P2_g ', 'Nivel')"].astype('category')
df["('P2_g ', 'Nivel')"] = df["('P2_g ', 'Nivel')"].cat\
                          .reorder_categories(['Júnior', 'Pleno', 'Sênior', 'Gestor'])

df["('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"] =\
    df["('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"].astype('category')

df["('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"] = \
    df["('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"].cat.reorder_categories([
                                                         '0',
                                                         '1 - 3',
                                                         '4 - 10',
                                                         '11 - 20',
                                                         '21 - 50', 
                                                         '51 - 100',
                                                         '101 - 300',
                                                         'Acima de 300 pessoas'                              
                                                            ])


# criando variavel "desfecho" para ser utilizada nos gráficos a seguir
desfecho = "('P1_b ', 'Genero')"


In [ ]:
#Função que cria gráficos de linha acoplado a gráfico de barra que mostra a diferença entre os grupos

def faz_grafico_diferenca_propocao(df, variavel, desfecho, titulo):
    fig = make_subplots(rows=2, cols=1, row_heights = [3,5], 
                        specs=[[{"secondary_y":True}],[{"secondary_y":True}]],
                        print_grid=False,
                        shared_xaxes="rows" )

    prop_genero = pd.crosstab(df[variavel], df[desfecho], normalize='columns')*100
    novo_df = pd.DataFrame(prop_genero['Feminino'] - prop_genero['Masculino'], columns=['diff'])
    linhas_2 = px.bar(novo_df, x=novo_df.index, y='diff', color_discrete_sequence = ['#710c04','lightgray'])

    linhas_2.update_layout(xaxis_showticklabels=False)
    for d in linhas_2.data:
        fig.add_trace((go.Bar(x=d['x'], y=d['y'], name = "Diferença <br />(Feminino - Masculino)", marker_color=np.where(novo_df["diff"]>0, '#710c04','lightgray'))), row=1, col=1)

    prop_genero = pd.crosstab(df[variavel],df[desfecho],normalize='columns')*100
    linhas_1 = px.line(prop_genero, x=prop_genero.index, y=['Feminino','Masculino'],
                  color_discrete_sequence = ['#710c04','lightgray' ])

    for d in linhas_1.data:
        fig.add_trace((go.Scatter(x=d['x'], y=d['y'], name = d['name'], line=d['line'], yaxis=d['yaxis'])), row=2, col=1)


    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')   
    fig.update_layout(xaxis_showticklabels=False)

    fig['layout']['title']= titulo
    fig['layout']['yaxis']['title']='Diferença entre<br /> proporções (%)'
    fig['layout']['yaxis3']['title']='Proporção (%)'
    fig.show()
    

# Diferenças salariais

Para avaliar as diferenças salariais, é apresentada, inicialmente,  uma visualização no estilo de pirâmide etária, utilizando as faixas salariais. 

Com essa visualização é possivel enxergar a variação de pessoas em cada faixa de salário e como cada gênero é representado quantitativamente.

Fica bem claro que a presença absoluta feminina em todas as faixas é consideravelmente menor, mas ela quase desaparece nas 3 faixas mais altas. Assim, além de exemplificar a diferença salarial, a pirâmide também deixa claro a baixa representatividade de mulheres no mundo de dados. Nessa pesquisa, o total de respondentes mulheres foi de 493, o que representa 18,7% dos respondentes (como pode ser visto na tabela abaixo do gráfico de pirâmide).








In [ ]:
#criando gráfico de pirâmide para salários

abs_genero = pd.crosstab(df["faixa_salarial"],df["('P1_b ', 'Genero')"])

women_pop = list(abs_genero.Feminino)
men_pop   = list(abs_genero.Masculino)
men_pop = [element * 1 for element in men_pop]

faixa = list(abs_genero.index.values)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=faixa,
    x=men_pop,
    name='Masculino',
    marker_color='lightgray',
    orientation='h'
))
fig.add_trace(go.Bar(
    y=faixa,
    x=women_pop,
    name='Feminino',
    marker_color='#710c04',
    orientation='h'
))

fig['layout']['title']= "Pirâmide salarial segundo o gênero"
fig['layout']['yaxis']['title']='Faixa salarial'

fig.update_layout(barmode='relative', xaxis_tickangle=-90)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.update_layout( width=800, height=500)
fig.show()

In [ ]:
#contruindo tabelas de frequências absolutas e percentuais 
# a partir da concatenação de duas crosstabs (frequencia absoluta e percentual)

tab = pd.concat(
[pd.crosstab(index=df["('P1_b ', 'Genero')"], columns = 'count').transpose(),
pd.crosstab(index=df["('P1_b ', 'Genero')"], columns = 'count',normalize=True).transpose()*100]).T

# mudando o nome das colunas 
tab.set_axis(['Freq',"%"],axis=1,inplace=True)

tab.index.name = 'Gênero'

tab


A diferença salarial fica ainda mais clara quando ignoramos os números absolutos e vemos as proporções dentro dos grupos (gêneros).

Para essa observação, são apresentados dois gráficos. Embaixo, um gráfico de linhas que apresentas as proporções salariais dentro de cada genêro e, acima dele, um gráfico de barras que mostra a diferença entre essas propoções. Quando a diferença é positiva, indica que as mulheres tem uma maior proporção do que homens naquela faixa salarial.

Nesse caso, vemos que a curva de salário segue o esperado: valores muito baixos e muito altos são pouco frequentes, e a maioria das pessoas recebe valores intermediários.

Um ponto de atenção nesse gráfico é que que a maior diferença entre os gêneros ocorre na faixa de 12 a 16 mil reais por mês, onde há uma diferença de 4% (7,8% das mulheres está nessa faixa, contra 11,9% dos homens). Um ponto interessante é que, juridicamente se define a faixa salarial acima de 12.867,14 como ["Trabalhadores Hipersuficientes"](https://www.conjur.com.br/2021-dez-25/trt-empregado-nao-diploma-hipersuficiente) e é exatamente nessa faixa que os homens passam a superar as mulheres


In [ ]:
faz_grafico_diferenca_propocao(df, 'faixa_salarial', desfecho, 
                               'Comparação entre as proproções de faixas salariais dentro de cada gênero')

Esse resultado nos leva a seguinte pergunta: 

**Existem diferenças objetivas entre homens e mulheres que justifiquem a disparidade salarial? Ou seja, é possível encontrar fatores como formação, tempo de experiência, entre outros, que auxiliem a explicar a disparidade?**

# O efeito tesoura: homens têm cargos em maior nível


Ao longo da análise, conforme as diferenças de gênero são investigadas, podemos encontrar semelhanças entre o mundo de dados e outros ambientes em que as desigualadades de gênero são velhas conhecidas. Algumas dessas desigualdades são tão conhecidas que tem nomes, são as metáforas da desigualdade. Deste modo, apontaremos quando os dados desse estudo refletirem tais metáforas

Uma dessas metáforas  é o **"efeito tesoura"**. Esse nome se deve ao desenho do gráfico que apresenta o número ou propoção de pessoas por gênero conforme a carreira avança.

Quando observamos a proporção de cargos dentro de cada gêneros, conseguimos ver com clareza o efeito tesoura para o nível de senioridade por gênero: a proporção entre mulheres decai conforme o nível do cargo aumenta e o inverso acontece para homens.




In [ ]:
faz_grafico_diferenca_propocao(df, "('P2_g ', 'Nivel')", desfecho, 
                        'Comparação das proproções dos níveis de senioridade <br /> dentro de cada gênero')

# Homens tem maior nível de escolaridade ou tempo de experiência?

**Nível de escolaridade**

Quando se analisa o nível de escolaridade, vemos que a proporção entre os homens é levemente maior nos níveis de escolaridade mais baixa. A curva das mulheres passa a ser superior a dos homens nos níveis de escolaridade mais alta. 

Aqui podemos tratar de outra metáfora da desigualdade: **"o chão grudento"**. Essa metáfora fala da dificuldade das mulheres deslancharem nas suas carreiras por situações que as "grudam". Nesse caso, apesar de terem tempo de estudo igual ou superior aos homens, algo tem atrasado as mulheres de alcançar os salários mais altos.

In [ ]:
faz_grafico_diferenca_propocao(df, "('P1_h ', 'Nivel de Ensino')", desfecho, 
                        'Comparação das proproções dos níveis de escolaridade por gênero')

**Área de formação**

No gráfico abaixo são apresentadas as proporções de cada área de formação dentro dos gêneros. Observa-se por exemplo, uma grande diferença entre as proporções de homens e mulheres que fazem cursos relacionados à computação\engenharia de software\S.I.\T.I (44% dos homens vs 35 % das mulheres). Comportamento diferente acontece nas áreas de Estatística/Matematica e Marketing onde a proporção de mulheres supera a masculina.

In [ ]:
# Criando gráficos de barras horizontais

#criando tabela cruzada para ser criada como input do gráfico de barras
prop_form = pd.crosstab(df["('P1_i ', 'Área de Formação')"],df["('P1_b ', 'Genero')"],normalize='columns')\
            .sort_values(by='Feminino',ascending=True)*100

formacao = list(prop_form.index.values)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=formacao,
    x=prop_form['Masculino'] ,
    name='Masculino',
    marker_color='lightgray',
    orientation='h'
))
fig.add_trace(go.Bar(
    y=formacao,
    x=prop_form['Feminino'] ,
    name='Feminino',
    marker_color='#710c04',
    orientation='h'
))

fig.update_layout(title="Proporção cada categoria de formação entre homens e mulheres")
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout( width=900, height=500)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.show()

De maneira complementar, no gráfico abaixo podemos observar também os percentuais de gênero dentro de cada área. Nesse caso, as áreas com maior prevalência feminina ficam ainda mais claras. Sendo que algumas áreas superam a proporção geral de mulheres (acima da linha preta), são elas: estatística, marketing e cinecias biológicas e relacionadas.

In [ ]:
# criando tabela  com as frequencias percentuais em cada genero  
# para poder obter o percentual feminina amostra e utilizá-lo como referencial no gráfico
tab_genero = pd.crosstab(df["('P1_b ', 'Genero')"],columns='count',normalize='columns')*100

media_participacao_feminina =  tab_genero.loc['Feminino']['count']


#criando tabela cruzada formação vs genero para ser criada como input do gráfico de barras
prop_form = pd.crosstab(df["('P1_i ', 'Área de Formação')"],df["('P1_b ', 'Genero')"],normalize='index')\
            .sort_values(by='Feminino',ascending=False)*100

formacao = list(prop_form.index.values)



fig = go.Figure()
fig.add_trace(go.Bar(
    y=formacao,
    orientation='h',
    x=prop_form['Feminino'] ,
    name='Feminino',
    marker_color='#710c04',
))
fig.add_trace(go.Bar(
    y=formacao,
    x=prop_form['Masculino'] ,
    name='Masculino',
    marker_color='lightgray',
    orientation='h'
))

fig.update_layout(title="Proporção de homens e mulheres dentro de cada categoria de formação")
fig.update_layout(barmode='stack', xaxis_tickangle=-45)
fig.update_layout( width=1000, height=400)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.add_vline(x=media_participacao_feminina,y1=0.85,line_color="black")
fig.add_annotation(x=media_participacap_feminina, y=8,
            text="Media participação feminina",showarrow=True,yshift=10,arrowhead=1)
fig.show()

**Tempo de experiência**

Quando falamos em tempo de experiência em dados, entre as mulheres existe uma proporção maior das que tem menos tempo de experiência.

Podemos fazer uma analogia aqui mais uma metáfora da desiguadade conhecida como "cano furado". Essa metáfora fala da perda de pessoas ao longo dos anos, principalmente quando se relaciona a mulheres. Podemos fazer a seguinte reflexão: mulheres tem menor tempo de experiência pois passaram a ingressar mais recentemente ou por que não conseguem permanecer muito tempo e geram a situação definida na metáfora do cano furado?

Para responder essa pergunta, é importante que esses dados continuem a ser coletados periodicamente para que possamos avaliar se esse esse padrão se perpetua, ou se estamos conseguindo tapar esse buraco e manter mulheres experientes dentro da área.

In [ ]:
faz_grafico_diferenca_propocao(df, "('P2_i ', 'Quanto tempo de experiência na área de dados você tem?')", desfecho, 
                        'Comparação das proprorções dos tempos de experiência em dados<br /> dentro de cada gênero')

  Em relação a experiência previa em TI/Engenharia de software, os grupos são mais semelhantes. 
  Mesmo assim, há um destaque para maior proporção de mulheres sem experiência nesse quesito. 

  É possível levantar a hipótese que mulheres tenham sido mais atraídas para a área de dados antes de tentar algo em computação pura.

In [ ]:
faz_grafico_diferenca_propocao(df, "('P2_j ', 'Quanto tempo de experiência na área de TI/Engenharia de Software você teve antes de começar a trabalhar na área de dados?')",
                               desfecho, 
                               'Comparação das proprorções do tempo de experiência prévia em TI dentro de cada gênero')


# Quais são os cargos com maior representatividade feminina?

Um outro fator que pode influenciar na valorização do profissional é o cargo que ocupa. Por diversos motivos, as faixas salariais de alguns cargos são superiores a de outros. O gráfico abaixo mostra o percentual de homens e mulheres dentro de cada possível cargo. Vemos que as mulheres, em geral, superam o percentual geral da amostra (linha preta) em cargos de analista, já os homens são maioria em cargos relacionados à engenharia.

In [ ]:
#criando gráfico de barras empilhadas para cargos

prop_form = pd.crosstab(df["('P2_f ', 'Cargo Atual')"],df["('P1_b ', 'Genero')"],normalize='index')\
            .sort_values(by='Feminino',ascending=False)*100

cargo = list(prop_form.index.values)



fig = go.Figure()
fig.add_trace(go.Bar(
    y=cargo,
    orientation='h',
    x=prop_form['Feminino'] ,
    name='Feminino',
    marker_color='#710c04',
))
fig.add_trace(go.Bar(
    y=cargo,
    x=prop_form['Masculino'] ,
    name='Masculino',
    marker_color='lightgray',
    orientation='h'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(title="Proporção de gênero dentro de cada ocupação")
fig.update_layout(barmode='stack', xaxis_tickangle=-45)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.add_vline(x=media_participacap_feminina,y1=.85,line_color="black")
fig.add_annotation(x=media_participacap_feminina, y=8,
            text="Proporção geral de mulheres",showarrow=True,yshift=180,arrowhead=1)
fig.show()

# Porte das empresas em que trabalham

Um outro fator que pode interferir no salário é o porte das empresas em que esses funcionários trabalham. Pensando nisso, os gráficos abaixo comparam a quantidade de funcionários das empresas de homens e mulheres. 
Quando falamos em número de funcinários na empresa de forma geral, não é possível identificar uma diferença entre os gêneros. Quando falamos na quantidade de funcionários em dados, podemos destacar que existe uma proporção maior de homens trabalhando em empresa com 4 a 10 funcionários.

In [ ]:
fig = make_subplots(rows=1, cols=2)

var = "('P2_c ', 'Numero de Funcionarios')"


prop_genero = pd.crosstab(df[var],df["('P1_b ', 'Genero')"],normalize='columns')*100
g1 = px.line(prop_genero, x=prop_genero.index, y=['Feminino','Masculino'],
              color_discrete_sequence = ['#710c04','lightgray' ]
               )
for d in g1.data:
        fig.add_trace((go.Scatter(x=d['x'], y=d['y'], name = d['name'], line=d['line'], yaxis=d['yaxis'],showlegend=False)), row=1, col=1)
        

var = "('P3_a ', 'Qual o número aproximado de pessoas que atuam com dados na sua empresa hoje?')"


prop_genero = pd.crosstab(df[var],df["('P1_b ', 'Genero')"],normalize='columns')*100
g2 = px.line(prop_genero, x=prop_genero.index, y=['Feminino','Masculino'],
              color_discrete_sequence = ['#710c04','lightgray' ]
               )

for d in g2.data:
        fig.add_trace((go.Scatter(x=d['x'], y=d['y'], name = d['name'], line=d['line'], yaxis=d['yaxis'])), row=1, col=2)  
        



fig['layout']['title'] = "Proporção do número de funcionários geral (esquerda) e em dados(direita) em cada gênero"
fig['layout']['yaxis']['title']='Proporção(%)'
fig['layout']['yaxis2']['title']='Proporção(%)'
fig['layout']['xaxis']['title']='Número de funcionários em geral'
fig['layout']['xaxis2']['title']='Número de funcionários em dados'
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')  
fig['layout']['yaxis2']['range']=[0,35]
fig['layout']['yaxis']['range']=[0,35]
#fig.update_layout()

fig.show()




# Ferramentas e linguagens mais utilizadas

Neste ponto são analisadas as linguagens de programação mais citadas entre homens e mulheres, que também são um fator de possível influência no salário. Observa-se que a proporção de homens que cita Python é superior a de mulheres (41% de homens contra 32% das mulheres). Em contrapartida, as mulheres tem maior proporção das que citam SQL ou que não utilizam linguagem.

In [ ]:
lista_P4_e = ["('P1_b ', 'Genero')","('P4_e_c ', 'Python')",
"('P4_e_f ', 'Java')","('P4_e_m ', 'Javascript')",
#"('P4_e_g ', 'Julia')",
"('P4_e_j ', 'Scala')","('P4_e_d ', 'C/C++/C#')",
"('P4_e_e ', '.NET')","('P4_e_k ', 'Matlab')",
"('P4_e_l ', 'PHP')","('P4_e_b ', 'R ')",
"('P4_e_h ', 'SAS/Stata')","('P4_e_i ', 'Visual Basic/VBA')",
"('P4_e_n ', 'Não utilizo nenhuma linguagem')","('P4_e_a ', 'SQL')"]


tab = df[lista_P4_e].groupby("('P1_b ', 'Genero')").mean()

tab = tab.transpose()

linguagem =  ['Python', 'Java',
 'Javascript', 'Scala', 'C/C++/C#', '.NET', 'Matlab', 'PHP', 'R ',
 'SAS/Stata', 'Visual Basic/VBA', 'Não utilizo nenhuma linguagem','SQL']


fig = go.Figure()
fig.add_trace(go.Bar(
    x=linguagem,
    y=tab['Masculino'] ,
    name='Masculino',
    marker_color='lightgray',
    orientation='v'
))
fig.add_trace(go.Bar(
    x=linguagem,
    y=tab['Feminino'] ,
    name='Feminino',
    marker_color='#710c04',#'#bd1e1e',
    orientation='v'
))


fig.update_layout(barmode='group', xaxis_tickangle=-45)
#fig.add_hrect(y0=6.5, y1=8.5, line_width=0, fillcolor="gray", opacity=0.2)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.add_vrect(x0=-.5, x1=0.5,line_dash="dash", line_color="gray",line_width=1,
              annotation_text="Predominância masculina", 
              annotation_position="top left",
              annotation=dict(font_size=12))
fig.add_vrect(x0=8.5, x1=12.5,line_dash="dash", line_color="gray",line_width=1,
              annotation_text="Predominância feminina", 
              annotation_position="top left",
              annotation=dict(font_size=12))

fig.update_layout(title="Proporções das linguagens dentro de cada gênero")

fig.show()

# Maiores interesses

Outra característica no mercado de trabalho que pode ser influenciada pelo gênero, são os itens mais buscados pelas pessoas.
Historicamente, homens sabem negociar melhor seus salários e tem menos aversão a risco, o que facilita a troca de emprego considerando apenas a remuneração. 
Os dados da pesquisa confirmaram essa hipótese, visto que quando perguntados sobre os interessem mais homens citam remuneração e benefícios do que mulheres.

Por outro lado, as mulheres apresentam maior frequência do que homens no interesse por ambiente, propósito e qualidade dos gestores, além da flexibilidade do trabalho remoto. Tais pontos podem refletir a necessidade de mulheres estarem em ambientes seguros (o que infelizmente ainda pode ser um desafio). Além disso, a flexibilidade de trabalho também é um fator importante, visto que mulheres são responsáveis pelas tarefas domesticas e familiares, sem ter com quem dividir.

In [ ]:
lista_P2_o = ["('P1_b ', 'Genero')","('P2_o_a ', 'Remuneração/Salário')", "('P2_o_b ', 'Benefícios')",
       "('P2_o_f ', 'Oportunidade de aprendizado e trabalhar com referências na área')",
       "('P2_o_j ', 'Reputação que a empresa tem no mercado')",
       "('P2_o_h ', 'Maturidade da empresa em termos de tecnologia e dados')",
       "('P2_o_g ', 'Plano de carreira e oportunidades de crescimento profissional')",
       "('P2_o_i ', 'Qualidade dos gestores e líderes')",
       "('P2_o_d ', 'Flexibilidade de trabalho remoto')",
       "('P2_o_c ', 'Propósito do trabalho e da empresa')",
       "('P2_o_e ', 'Ambiente e clima de trabalho')"]


tab = df[lista_P2_o].groupby("('P1_b ', 'Genero')").mean()

tab = tab.transpose()

tab

motivacoes = ['Remuneração','Benefícios','Oportunidades','Reputação',\
              'Maturidade tech','Plano de carreira', 'Qualidade gestores','Flexibilidade remoto',\
             'Propósito', 'Ambiente']


fig = go.Figure()
fig.add_trace(go.Bar(
    x=motivacoes,
    y=tab['Masculino'] ,
    name='Masculino',
    marker_color='lightgray',
    orientation='v'
))
fig.add_trace(go.Bar(
    x=motivacoes,
    y=tab['Feminino'] ,
    name='Feminino',
    marker_color='#710c04',
    orientation='v'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
#fig.add_hrect(y0=6.5, y1=8.5, line_width=0, fillcolor="gray", opacity=0.2)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')
fig.add_vrect(x0=-.5, x1=2.5,line_dash="dash", line_color="gray",line_width=1,
              annotation_text="Predominância masculina", 
              annotation_position="top right",
              annotation=dict(font_size=18))
fig.add_vrect(x0=5.5, x1=9.5,line_dash="dash", line_color="gray",line_width=1,
              annotation_text="Predominância feminina", 
              annotation_position="top left",
              annotation=dict(font_size=18))

fig.update_layout(title="Proporções dos maiores interesses em cada gênero")

fig.show()

# Pontos principais da análise exploratória

Em resumo, a análise exploratória realizada mostra que ainda que mulheres tenham, proporcionalmente, maiores níveis educacionais do que os homens, existe uma disparidade salarial e dos níveis de cargos ocupados por homens e mulheres. É possível que essa disparidade seja explicada, por exemplo, pelo maior tempo de experiência ou formação em carreiras ligadas à computação entre os homens, entre outros.

Sabemos que variáveis confundidoras como formação, tempo de experiência, nível educacional, cargo podem afetar diretamente os níveis salariais e cargos ocupados.  Por isso, as diferenças entre homens e mulheres nessas variáveis dificultam o entendimento de quais são os causadores da disparidade salarial. Na prática, gostaríamos de entender: 

- Se os grupos fossem semelhantes em relação à variáveis confundidoras, a disparidade salarial deixaria de existir?
- O padrão de definição de salários para homens é o mesmo seguido para mulheres?


Mesmo que os resultados acima sejam otimistas, vemos que os fatores mais importantes na predição de salário são a senioridade do cargo, tempo de experiência e ocupar cargos relacionados à engenharia. Na análise exploratória, vimos que esses são pontos de difícil acesso para as mulheres. Podemos inferir, então, que a disparidade salarial se deve à dificuldade que as mulheres tem de acessarem cargos mais altos, formações/ocupações ligadas à computação ou terem tempo de experiência equivalente.

Sendo assim, podemos falar em uma última métafora da desigualdade: **a metáfora do teto de vidro**, que é definida pela dificuldade de mulheres alcançarem cargos mais altos, apesar de não existirem barreiras visíveis (objetivas) que as impeçam. 


Um outro ponto importante a se destacar é que esse estudo considerou faixas amplas de salário (de 4 em 4 mil), o que pode maquear algumas diferenças que possam estar ocorrendo dentro dessas faixas. Em outras palavras, é possível que um grupo esteja mais localizado nas bordas superiores de uma faixa, enquanto o outro esta nas bordas inferiores.
Nesse sentido, poderíamos testar outros modelos e avaliar se os resultados se mantêm. Se soubessemos os valores absolutos de salário (ou algo próximo disso), por exemplo, será que os erros para mais e para menos teriam um padrão?





## Conclusão

- O cenário pode ser considerado positivo, visto que em condições de igualdade é provavável que haja equivalência salarial. 
- A análise exploratória indica que **ainda não estamos em codições de igualdade**, por isso é necessário:
> - Criar ambientes agradáveis que permitam a permanência das mulheres
> - Criar ações propositivas para que elas alcancem cargos de maior senioridade
> - Manter e intensificar as ações afirmativas de atração de mulheres para a área de dados e de meninas para cursos de graduação ligados a dados. 



